In [6]:
!ls -la

total 20
drwxr-xr-x 1 root root 4096 May  6 14:23 .
drwxr-xr-x 1 root root 4096 May  6 14:21 ..
drwxr-xr-x 1 root root 4096 Apr 29 16:32 .config
drwxr-xr-x 3 root root 4096 May  6 14:24 embeddings
drwxr-xr-x 1 root root 4096 Apr 29 16:32 sample_data


In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 63kB/s 
     |████████████████████████████████| 419kB 50.6MB/s 
     |████████████████████████████████| 61kB 27.5MB/s 
     |████████████████████████████████| 3.0MB 49.9MB/s 


**Step 1:  GloVe as Tensorflow embedding layer**

In [0]:
!pip install chakin

In [7]:
import numpy as np
import tensorflow as tf
import chakin

import json
import os
from collections import defaultdict

chakin.search(lang='English')
#14        GloVe.6B.300d        300  Wikipedia+Gigaword 5 (6B)           400K
# Downloading GloVe.6B.300d embeddings from Stanford:
CHAKIN_INDEX = 14
NUMBER_OF_DIMENSIONS = 300
SUBFOLDER_NAME = "glove.6B"

DATA_FOLDER = "embeddings"
ZIP_FILE = os.path.join(DATA_FOLDER, "{}.zip".format(SUBFOLDER_NAME))
ZIP_FILE_ALT = "glove" + ZIP_FILE[5:]  # sometimes it's lowercase only...
UNZIP_FOLDER = os.path.join(DATA_FOLDER, SUBFOLDER_NAME)
if SUBFOLDER_NAME[-1] == "d":
    GLOVE_FILENAME = os.path.join(UNZIP_FOLDER, "{}.txt".format(SUBFOLDER_NAME))
else:
    GLOVE_FILENAME = os.path.join(UNZIP_FOLDER, "{}.{}d.txt".format(SUBFOLDER_NAME, NUMBER_OF_DIMENSIONS))


if not os.path.exists(ZIP_FILE) and not os.path.exists(UNZIP_FOLDER):
    # GloVe by Stanford is licensed Apache 2.0: 
    #     https://github.com/stanfordnlp/GloVe/blob/master/LICENSE
    #     http://nlp.stanford.edu/data/glove.6B.zip
    #     Copyright 2014 The Board of Trustees of The Leland Stanford Junior University
    print("Downloading embeddings to '{}'".format(ZIP_FILE))
    chakin.download(number=CHAKIN_INDEX, save_dir='./{}'.format(DATA_FOLDER))
else:
    print("Embeddings already downloaded.")
    
if not os.path.exists(UNZIP_FOLDER):
    import zipfile
    if not os.path.exists(ZIP_FILE) and os.path.exists(ZIP_FILE_ALT):
        ZIP_FILE = ZIP_FILE_ALT
    with zipfile.ZipFile(ZIP_FILE,"r") as zip_ref:
        print("Extracting embeddings to '{}'".format(UNZIP_FOLDER))
        zip_ref.extractall(UNZIP_FOLDER)
else:
    print("Embeddings already extracted.")


                   Name  Dimension                     Corpus VocabularySize  \
2          fastText(en)        300                  Wikipedia           2.5M   
11         GloVe.6B.50d         50  Wikipedia+Gigaword 5 (6B)           400K   
12        GloVe.6B.100d        100  Wikipedia+Gigaword 5 (6B)           400K   
13        GloVe.6B.200d        200  Wikipedia+Gigaword 5 (6B)           400K   
14        GloVe.6B.300d        300  Wikipedia+Gigaword 5 (6B)           400K   
15       GloVe.42B.300d        300          Common Crawl(42B)           1.9M   
16      GloVe.840B.300d        300         Common Crawl(840B)           2.2M   
17    GloVe.Twitter.25d         25               Twitter(27B)           1.2M   
18    GloVe.Twitter.50d         50               Twitter(27B)           1.2M   
19   GloVe.Twitter.100d        100               Twitter(27B)           1.2M   
20   GloVe.Twitter.200d        200               Twitter(27B)           1.2M   
21  word2vec.GoogleNews        300      

In [0]:
print(tf.__version__)
!ls -la embeddings/glove.6B

2.0.0-alpha0
total 2197156
drwxr-xr-x 2 root root       4096 May  6 02:39 .
drwxr-xr-x 3 root root       4096 May  6 02:39 ..
-rw-r--r-- 1 root root  347116733 May  6 02:39 glove.6B.100d.txt
-rw-r--r-- 1 root root  693432828 May  6 02:39 glove.6B.200d.txt
-rw-r--r-- 1 root root 1037962819 May  6 02:39 glove.6B.300d.txt
-rw-r--r-- 1 root root  171350079 May  6 02:39 glove.6B.50d.txt


In [0]:
def load_embedding_from_disks(glove_filename, with_indexes=True):
    """
    Read a GloVe txt file. If `with_indexes=True`, we return a tuple of two dictionnaries
    `(word_to_index_dict, index_to_embedding_array)`, otherwise we return only a direct 
    `word_to_embedding_dict` dictionnary mapping from a string to a numpy array.
    """
    if with_indexes:
        word_to_index_dict = dict()
        index_to_embedding_array = []
    else:
        word_to_embedding_dict = dict()

    
    with open(glove_filename, 'r') as glove_file:
        for (i, line) in enumerate(glove_file):
            
            split = line.split(' ')
            
            word = split[0]
            
            representation = split[1:]
            representation = np.array(
                [float(val) for val in representation]
            )
            
            if with_indexes:
                word_to_index_dict[word] = i
                index_to_embedding_array.append(representation)
            else:
                word_to_embedding_dict[word] = representation

    _WORD_NOT_FOUND = [0.0]* len(representation)  # Empty representation for unknown words.
    if with_indexes:
        _LAST_INDEX = i + 1
        word_to_index_dict = defaultdict(lambda: _LAST_INDEX, word_to_index_dict)
        index_to_embedding_array = np.array(index_to_embedding_array + [_WORD_NOT_FOUND])
        return word_to_index_dict, index_to_embedding_array
    else:
        word_to_embedding_dict = defaultdict(lambda: _WORD_NOT_FOUND)
        return word_to_embedding_dict



In [9]:
print("Loading embedding from disks...")
word_to_index, index_to_embedding = load_embedding_from_disks(GLOVE_FILENAME, with_indexes=True)
print("Embedding loaded from disks.")

vocab_size, embedding_dim = index_to_embedding.shape
print("Embedding is of shape: {}".format(index_to_embedding.shape))
print("This means (number of words, number of dimensions per word)\n")


Loading embedding from disks...
Embedding loaded from disks.
Embedding is of shape: (400001, 300)
This means (number of words, number of dimensions per word)



**Load the pre-trained GloVe embedding in TensorFlow**

In [0]:
batch_size = 4000 # Total Vocabulary’s size is 400K words

# Define the variable that will hold the embedding:
tf_embedding = tf.Variable(
    tf.constant(0.0, shape=index_to_embedding.shape),
    trainable=False,
    name="tf_embedding"
)

tf_word_ids = tf.Variable(tf.constant(0,shape=[batch_size]),name="tf_word_ids")
tf_word_representation_layer = tf.nn.embedding_lookup(
    params=tf_embedding,
    ids=tf_word_ids
)


**Sending the embedding to TensorFlow**

In [0]:
tf_embedding = index_to_embedding

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self,embeddinglist):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)
    self.bias = embeddinglist

  def call(self, x):
    return self.l1(x)


**Save for later use**

In [0]:
prefix = SUBFOLDER_NAME + "." + str(NUMBER_OF_DIMENSIONS) + "d"
TF_EMBEDDINGS_FILE_NAME = os.path.join(DATA_FOLDER, prefix + ".ckpt")
DICT_WORD_TO_INDEX_FILE_NAME = os.path.join(DATA_FOLDER, prefix + ".json")

variables_to_save = [tf_embedding]
net = Net(tf_embedding)
ckpt = tf.train.Checkpoint(net=net)
manager = tf.train.CheckpointManager(ckpt, DATA_FOLDER,max_to_keep=5)
save_path = manager.save()
#print("Testing value 3366 from tf_embedding {}".format(tf_embedding[3366]))
print("TF embeddings saved to '{}'.".format(save_path))

with open(DICT_WORD_TO_INDEX_FILE_NAME, 'w') as f:
    json.dump(word_to_index, f)
print("word_to_index dict saved to '{}'.".format(DICT_WORD_TO_INDEX_FILE_NAME))


In [0]:
!ls -la embeddings

total 841992
drwxr-xr-x 3 root root      4096 May  3 14:53 .
drwxr-xr-x 1 root root      4096 May  3 14:52 ..
drwxr-xr-x 2 root root      4096 May  3 14:54 glove.6B
-rw-r--r-- 1 root root 862182613 May  3 14:53 glove.6B.zip


**Reset the Jupyter notebook and load the embedding from disk**

In [0]:
# Delete variables and restart the Python kernel
#%reset


In [0]:
import numpy as np
import tensorflow as tf
import os
import json
from string import punctuation
from collections import defaultdict
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import time

batch_size = 100 # Any size is accepted
word_representations_dimensions = 300  

DATA_FOLDER = "embeddings"
SUBFOLDER_NAME = "glove.6B"
TF_EMBEDDING_FILE_NAME = "{}.ckpt".format(SUBFOLDER_NAME)
SUFFIX = SUBFOLDER_NAME + "." + str(word_representations_dimensions)
TF_EMBEDDINGS_FILE_PATH = os.path.join(DATA_FOLDER, SUFFIX + "d.ckpt")
DICT_WORD_TO_INDEX_FILE_NAME = os.path.join(DATA_FOLDER, SUFFIX + "d.json")

In [0]:
def load_word_to_index(dict_word_to_index_file_name):
    """
    Load a `word_to_index` dict mapping words to their id, with a default value
    of pointing to the last index when not found, which is the unknown word.
    """
    with open(dict_word_to_index_file_name, 'r') as f:
        word_to_index = json.load(f)
    _LAST_INDEX = len(word_to_index) - 2  # Why - 2? Open issue?
    print("word_to_index dict restored from '{}'.".format(dict_word_to_index_file_name))
    word_to_index = defaultdict(lambda: _LAST_INDEX, word_to_index)

    return word_to_index

def load_embedding_tf(word_to_index, tf_embeddings_file_path, nb_dims):
    """
    Define the embedding tf.Variable and load it.
    """
   
    tf_embedding = tf.Variable(tf.zeros(shape=(len(word_to_index)+1, nb_dims)))
    #print("Shape of the variable to restore = {}".format(tf_embedding.shape))  
    ckpt = tf.train.Checkpoint(var=tf_embedding)   
    manager = tf.train.CheckpointManager(ckpt, DATA_FOLDER, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)   
    print("TF 2.0 embeddings restored from '{}'.".format(manager.latest_checkpoint))
    #print("embedding restored has shape={}".format(tf_embedding.shape))
    #print("load_embedding_tf: Test restored value 3366 from tf_embedding {}".format(tf_embedding[3366]))
    return tf_embedding


**Restore the embeddings from disk**

In [0]:
!ls -la embeddings

total 1318264
drwxr-xr-x 3 root root      4096 May  3 07:12 .
drwxr-xr-x 1 root root      4096 May  3 06:41 ..
-rw-r--r-- 1 root root       166 May  3 07:12 checkpoint
-rw-r--r-- 1 root root 480001398 May  3 07:12 ckpt-1.data-00000-of-00001
-rw-r--r-- 1 root root       271 May  3 07:12 ckpt-1.index
drwxr-xr-x 2 root root      4096 May  3 06:43 glove.6B
-rw-r--r-- 1 root root   7691463 May  3 07:12 glove.6B.300d.json
-rw-r--r-- 1 root root 862182613 May  3 06:43 glove.6B.zip


In [0]:
# Load the embedding matrix in tf
word_to_index = load_word_to_index(
    DICT_WORD_TO_INDEX_FILE_NAME)
tf_embedding = load_embedding_tf(
    word_to_index,
    TF_EMBEDDINGS_FILE_PATH, 
    word_representations_dimensions)

In [0]:
#longtext can be full text of the paper (max = 5000 words) or the abstract (250 words)
def text_to_word_ids(longtext, word_to_index):
    """
    Note: there might be a better way to split sentences for GloVe.
    Please look at the documentation or open an issue to suggest a fix.
    """
    # Separating punctuation from words:
    for punctuation_character in punctuation:
        longtext = longtext.replace(punctuation_character, " {} ".format(punctuation_character))
    # Removing double spaces and lowercasing:
    longtext = longtext.replace("  ", " ").replace("  ", " ").lower().strip()
    # Splitting on every space:
    split_longtext = longtext.split(" ")
    # Converting to IDs:
    ids = [word_to_index[w.strip()] for w in split_longtext]
    return ids, split_longtext
def ids_to_embeddings(wordids):
    embeddings_tensor = [tf_embedding[wordid] for wordid in wordids]
    return embeddings_tensor
def ids_to_embeddings_V2(wordids,padToSize):
    # no padding operation    
    if padToSize <= 0:
      embeddings_tensor = [tf_embedding[wordid] for wordid in wordids]
    else:
      wordlistLength = len (wordids)      
      remainNumberToPad = padToSize - wordlistLength
      if remainNumberToPad < 0:
        # wordList is already longer than padToSize => Trim to padToSize
        for wordid in wordids:
          embeddings_tensor.extend(tf_embedding[wordid])
      elif remainNumberToPad == 0:
        #wordList equals padToSize then do as no padding
        embeddings_tensor = [tf_embedding[wordid] for wordid in wordids]
      else:
        #wordList is less than padToSize then do the padding        
        tempList = []
        for i in range(0,wordlistLength):
            tempList.append(wordids[i])
        for j in range(wordlistLength,padToSize):
            #pad the UNKNOW_WORD index
            tempList.append(400000) 
        embeddings_tensor = [tf_embedding[wordid] for wordid in tempList]       
    return embeddings_tensor  

**Test function Text2WordIDs:**

In [0]:
strFullPaper = "The world is beautiful and today is a nice day!"
strAbstract = "Beautiful and nice day."

word_ids_list, resulttext = text_to_word_ids(strAbstract,word_to_index)

print("Text to vector with content = \"{}\":".format(strAbstract))

for word, wordid in zip(resulttext, word_ids_list):
	print("     {}{}".format((word+ ":").ljust(15),str(wordid)))
  
test_abstract_embedding = ids_to_embeddings(word_ids_list)
print("test_abstract_embedding len={}".format(len(test_abstract_embedding)))
print("test_abstract_embedding has shape = {}".format(len(test_abstract_embedding)))
print("test_abstract_embedding, each word is a tensor = {}".format(test_abstract_embedding[0]))
print("test_abstract_embedding, each word has shape = {}".format(test_abstract_embedding[0].shape))
print("test_abstract_embedding, each word has dtype = {}".format(test_abstract_embedding[0].dtype))


**Load sample pair of 2 text files (FullPaper.txt; Abstract.txt) using DataSet**

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls -la "drive/My Drive/Tensorflow/Data/"

total 12
drwx------ 2 root root 4096 May  2 03:31 DataSet
drwx------ 2 root root 4096 Apr 26 09:58 Testing
drwx------ 2 root root 4096 Apr 26 09:59 Training


In [56]:
DATASET_FOLDER = "drive/My Drive/Tensorflow/Data/DataSet"

TRAIN_DATA_FOLDER = "drive/My Drive/Tensorflow/Data/Training"
TRAIN_DATA_FILE = "Training.zip"
TRAIN_DATA_FILE_PATH = os.path.join(DATASET_FOLDER, TRAIN_DATA_FILE)

TEST_DATA_FOLDER = "drive/My Drive/Tensorflow/Data/Testing"
TEST_DATA_FILE = "Testing.zip"
TEST_DATA_FILE_PATH = os.path.join(DATASET_FOLDER, TEST_DATA_FILE)
import zipfile

UNZIP_DATA_FOLDER = TRAIN_DATA_FOLDER
with zipfile.ZipFile(TRAIN_DATA_FILE_PATH,"r") as zip_ref:
        print("Extracting training data to '{}'".format(UNZIP_DATA_FOLDER))
        zip_ref.extractall(UNZIP_DATA_FOLDER)
      
with zipfile.ZipFile(TEST_DATA_FILE_PATH,"r") as zip_ref:
        print("Extracting testing data to '{}'".format(TEST_DATA_FOLDER))
        zip_ref.extractall(TEST_DATA_FOLDER)


Extracting training data to 'drive/My Drive/Tensorflow/Data/Training'
Extracting testing data to 'drive/My Drive/Tensorflow/Data/Testing'


In [57]:
!ls -la "drive/My Drive/Tensorflow/Data/Testing"

total 28
-rw------- 1 root root   898 May  6 15:59 A-1.txt
-rw------- 1 root root   534 May  6 15:59 A-2.txt
-rw------- 1 root root 18459 May  6 15:59 F-1.txt
-rw------- 1 root root  7264 May  6 15:59 F-2.txt


**Load the text files**

In [0]:
# Load txt files: Please download the Glove dataset (file's size = 330GB) from its website 
# And put the (subset) data files (trying with subset/smaller dataset) in Google Drive as zip file
# F[0-9].txt: the full paper text files
# A[0-9].txt: the abstract text files
def dataset_input_fn():
  fullpaper_filenames = ["drive/My Drive/Tensorflow/Data/Training/F-1.txt","drive/My Drive/Tensorflow/Data/Training/F-2.txt"]
  abstract_filenames = ["drive/My Drive/Tensorflow/Data/Training/A-1.txt","drive/My Drive/Tensorflow/Data/Training/A-2.txt"]
  dataset_fullpapers = tf.data.Dataset.from_tensor_slices(fullpaper_filenames)
  dataset_abstracts = tf.data.Dataset.from_tensor_slices(abstract_filenames)

  dataset_fullpapers = dataset_fullpapers.map(
    lambda filename: (
        tf.io.read_file(filename)))
  dataset_abstracts = dataset_abstracts.map(lambda filename: (
        tf.io.read_file(filename)))
  for data in dataset_abstracts:
    print(data.shape)
  # Zip the sentence and the labels together
  training_dataset = tf.data.Dataset.zip((dataset_fullpapers, dataset_abstracts))
  return training_dataset

In [0]:
training_dataset = dataset_input_fn() 

In [0]:
!ls -la embeddings

total 841992
drwxr-xr-x 3 root root      4096 May  3 14:53 .
drwxr-xr-x 1 root root      4096 May  3 15:00 ..
drwxr-xr-x 2 root root      4096 May  3 14:54 glove.6B
-rw-r--r-- 1 root root 862182613 May  3 14:53 glove.6B.zip


In [0]:
print(tf.__version__)

2.0.0-alpha0


In [0]:
#datasetelement: each element in the dataset is a pair of string(fullpaper,abstract).
#The number of elements in the dataset is the number of papers. In the testing, there are 2 papers => dataset has 2 elements.
#then we call the function to transform:
#The fullpaper (max 5000 words) => 5000 x 300 = Tensor (5000,300)
#and abstract (max 250 words) => 250 x 300 = Tensor (250,300)
#into matrix of embedding vectors (300 dimensions each vector), one vector per word.
def prepare_embedding_dataset_fn(papertext,abstracttext):   
  wordids_fullpaper,_ = text_to_word_ids(format(papertext),word_to_index) 
  fullpaper_embedding = ids_to_embeddings(wordids_fullpaper)  
  wordids_abstract,_ = text_to_word_ids(format(abstracttext),word_to_index)  
  abstract_embedding = ids_to_embeddings(wordids_abstract)
 
  return fullpaper_embedding,abstract_embedding

In [0]:
for papertext,abstracttext in training_dataset:
    fullpaper_embedding,abstract_embedding = prepare_embedding_dataset_fn(papertext,abstracttext) 

**Create the models**

**Conditional GAN's**

In [0]:
BUFFER_SIZE = 400
BATCH_SIZE = 1

In [0]:
def downsample(filters, size, apply_batchnorm=True):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

**Testing**

In [0]:
down_model = downsample(300, 4)
for fullpaper,abstract in training_dataset:
  word_ids_list, resulttext = text_to_word_ids(format(fullpaper),word_to_index)
  abstract_word_ids_list, resulttextabstract = text_to_word_ids(format(abstract),word_to_index)
  print("word_ids_list length ={}".format(len(word_ids_list)))
  # 4096 words x 300 dimension embedding vector => Tensor [64,64,300]
  test_fullpaper_embedding = ids_to_embeddings_V2(word_ids_list,4096)
  test_abstract_embedding = ids_to_embeddings_V2(abstract_word_ids_list,256)
  print("test_fullpaper_embedding length={}".format(len(test_fullpaper_embedding)))
 
  test = tf.convert_to_tensor(test_fullpaper_embedding)
  print("test_fullpaper_embedding convert to tensor with dtype={}".format(test.dtype))
  test = tf.cast(test, tf.float32)
  print("test_fullpaper_embedding convert to tensor with with cast, dtype={}".format(test.dtype))
  testvector = tf.reshape(test,[64,64,300])
  print("testvector type = {}".format(testvector.dtype))
  print("testvector shape = {}".format(testvector.shape))
  test_input = tf.expand_dims(testvector, 0)
  print("test_input type = {}".format(test_input.dtype))
  print("test_input shape = {}".format(test_input.shape))
  down_result = down_model(test_input)
  print (down_result.shape)
    
  test_abstract_tensor = tf.convert_to_tensor(test_abstract_embedding)
  test_abstract_tensor = tf.cast(test_abstract_tensor, tf.float32)
  test_abstract_vector = tf.reshape(test_abstract_tensor,[16,16,300])
  test_abstract_vector_input = tf.expand_dims(test_abstract_vector, 0)
 
  down_abstract_model = downsample(300, 3)
  down_abstract_result = down_abstract_model(test_abstract_vector_input)
  print (down_abstract_result.shape)

In [0]:
def padWordIndex(word_ids_array,padToSize):
  resultarray = []
  currentwordlen = len (word_ids_array)
  if currentwordlen == padToSize: # no need to pad
    resultarray = word_ids_array
  elif currentwordlen > padToSize: # already greater, then trim down
    for i in range(0,padToSize):
      resultarray.append(word_ids_array[i])
  else: # pad up to the padToSize
    for i in range(0,currentwordlen):
      resultarray.append(word_ids_array[i])
    for j in range(currentwordlen,padToSize):
      resultarray.append(400000) # UNKNOW WORD INDEX
  return resultarray  
  

In [0]:
down_text = downsample(64,2)
for fullpaper,abstract in training_dataset:
  word_ids_list, resulttext = text_to_word_ids(format(fullpaper),word_to_index)
  testpadarray = padWordIndex(word_ids_list,4096)
  print(len(testpadarray))
  testpadarray = tf.cast(testpadarray,tf.float32)
  word_ids_vector = tf.reshape(testpadarray,[64,64,1])
  word_ids_vector = tf.expand_dims(word_ids_vector, 0)
  down_text_result = down_text(word_ids_vector)
  print(down_text_result.shape)

In [0]:
def upsample(filters, size, apply_dropout=False):
  initializer = tf.random_normal_initializer(0., 0.02)

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer=initializer,
                                    use_bias=False))
  
  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result

In [0]:
up_model = upsample(300, 4)
up_result = up_model(down_result)
print (up_result.shape)


(1, 64, 64, 300)


In [0]:
#number of embedding vector from Glove
EMBEDDING_DIMS = 300

In [0]:
def Generator():
  down_stack = [
    downsample(300, 4, apply_batchnorm=False), # (bs, 32, 32, 300)
    downsample(300, 4), # (bs, 16, 16, 300)
    downsample(300, 4), # (bs, 8, 8, 300)
    downsample(300, 4), # (bs, 4, 4, 300)
    downsample(300, 4), # (bs, 2, 2, 300)
    downsample(300, 4) # (bs, 1, 1, 300)
    #downsample(512, 4), # (bs, 2, 2, 512)
    #downsample(512, 4), # (bs, 1, 1, 512)
  ]

  up_stack = [
    upsample(300, 4, apply_dropout=True), # (bs, 2, 2, 300)
    upsample(300, 4, apply_dropout=True), # (bs, 4, 4, 300)
    upsample(300, 4, apply_dropout=True),    # (bs, 8, 8, 300)
    #upsample(300, 4), # (bs, 16, 16, 300)
    #upsample(300, 4), # (bs, 32, 32, 300)
    #upsample(300, 4) # (bs, 64, 64, 300)
    #upsample(64, 4), # (bs, 128, 128, 128)
  ]

  initializer = tf.random_normal_initializer(0., 0.02)
  last = tf.keras.layers.Conv2DTranspose(EMBEDDING_DIMS, 4,
                                         strides=(2,2),
                                         padding='same',
                                         dilation_rate=(1, 1),
                                         kernel_initializer=initializer,
                                         activation='tanh') # (bs, 16, 16, 300)

  concat = tf.keras.layers.Concatenate()

  inputs = tf.keras.layers.Input(shape=[None,None,300])
  x = inputs

  # Downsampling through the model
  skips = []
  for down in down_stack:
    x = down(x)
    skips.append(x)

  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    x = concat([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)


In [23]:
generator = Generator()
gen_output = generator(test_input, training=False)
print("gen_output shape ={}".format(gen_output.shape))

gen_output shape =(1, 16, 16, 300)


**Build the Discriminator**

In [0]:
def Discriminator():
  initializer = tf.random_normal_initializer(0., 0.02)

  inp = tf.keras.layers.Input(shape=[None, None, 300], name='input_abstract')
  tar = tf.keras.layers.Input(shape=[None, None, 300], name='target_abstract')

  x = tf.keras.layers.concatenate([inp, tar]) # (bs, 16, 16, 300)

  #down1 = downsample(300, 4, False)(x) # (bs, 8, 8, 300)
  #down2 = downsample(300, 4)(down1) # (bs, 4, 4, 300)
  #down3 = downsample(300, 4)(down2) # (bs, 2, 2, 300)

  #zero_pad1 = tf.keras.layers.ZeroPadding2D()(down3) # (bs, 4,4, 300)
  zero_pad1 = x
  conv = tf.keras.layers.Conv2D(300, 4, strides=1,
                                kernel_initializer=initializer,
                                use_bias=False)(zero_pad1) # (bs, 31, 31, 300)

  batchnorm1 = tf.keras.layers.BatchNormalization()(conv)

  leaky_relu = tf.keras.layers.LeakyReLU()(batchnorm1)

  zero_pad2 = tf.keras.layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 300)

  last = tf.keras.layers.Conv2D(1, 4, strides=1,
                                kernel_initializer=initializer)(zero_pad2) # (bs, 12, 12, 1)

  return tf.keras.Model(inputs=[inp, tar], outputs=last)


In [0]:
discriminator = Discriminator()
disc_out = discriminator([test_abstract_vector_input, gen_output], training=False)


In [27]:
print("disc_out:")
print(disc_out.shape)

disc_out:
(1, 12, 12, 1)


**Loss function and optimizer**

In [0]:
LAMBDA = 100
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [0]:
def generator_loss(disc_generated_output, gen_output, target):
  gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

  # mean absolute error
  l1_loss = tf.reduce_mean(tf.abs(target - gen_output))

  total_gen_loss = gan_loss + (LAMBDA * l1_loss)

  return total_gen_loss

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


In [0]:
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


In [0]:
EPOCHS = 20 #200

In [0]:
@tf.function
def train_step(input_fullpaper, target_abstract):
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    gen_output = generator(input_fullpaper, training=True)

    disc_real_output = discriminator([target_abstract, target_abstract], training=True)
    disc_generated_output = discriminator([target_abstract, gen_output], training=True)

    gen_loss = generator_loss(disc_generated_output, gen_output, target_abstract)
    disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

  generator_gradients = gen_tape.gradient(gen_loss,
                                          generator.trainable_variables)
  discriminator_gradients = disc_tape.gradient(disc_loss,
                                               discriminator.trainable_variables)

  generator_optimizer.apply_gradients(zip(generator_gradients,
                                          generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(discriminator_gradients,
                                              discriminator.trainable_variables))


In [0]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for input_fullpaper, target_abstract in dataset:
      embedding_fullpaper,embedding_abstract = get_embedding_pair(input_fullpaper,target_abstract)
      train_step(embedding_fullpaper, embedding_abstract)

    #clear_output(wait=True)
    #for inp, tar in test_dataset.take(1):
    #  generate_images(generator, inp, tar)

    # saving (checkpoint) the model every 20 epochs
    if (epoch + 1) % 20 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                        time.time()-start))


In [0]:
def get_embedding_pair(input_fullpaper,target_abstract):
  fullpaper_word_ids_list, resulttext = text_to_word_ids(format(input_fullpaper),word_to_index)
  abstract_word_ids_list, resulttextabstract = text_to_word_ids(format(target_abstract),word_to_index)
   
  fullpaper_embedding = ids_to_embeddings_V2(fullpaper_word_ids_list,4096)
  abstract_embedding = ids_to_embeddings_V2(abstract_word_ids_list,256)  
  
  fullpaper_embedding_tensor = tf.convert_to_tensor(test_fullpaper_embedding)   
  fullpaper_embedding_tensor = tf.cast(fullpaper_embedding_tensor, tf.float32)   
  fullpaper_output = tf.reshape(fullpaper_embedding_tensor,[64,64,300]) 
  fullpaper_output = tf.expand_dims(fullpaper_output, 0)
   
  
  abstract_tensor = tf.convert_to_tensor(abstract_embedding)
  abstract_tensor = tf.cast(abstract_tensor, tf.float32)
  abstract_tensor = tf.reshape(abstract_tensor,[16,16,300])
  abstract_output = tf.expand_dims(abstract_tensor, 0)
  
  return fullpaper_output,abstract_output

**Input pipeline**

In [0]:
BUFFER_SIZE = 256

In [0]:
def load_textfiles_train(fullpaper_filenames,abstract_filenames):
  #fullpaper_filenames = ["drive/My Drive/Tensorflow/Data/Training/F-1.txt","drive/My Drive/Tensorflow/Data/Training/F-2.txt"]
  #abstract_filenames = ["drive/My Drive/Tensorflow/Data/Training/A-1.txt","drive/My Drive/Tensorflow/Data/Training/A-2.txt"] 

  dataset_fullpapers = fullpaper_filenames.map(
    lambda filename: (
        tf.io.read_file(filename)),num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset_abstracts = abstract_filenames.map(lambda filename: (
        tf.io.read_file(filename)),num_parallel_calls=tf.data.experimental.AUTOTUNE)
   
  # Zip the full paper and the abstract together
  training_dataset = tf.data.Dataset.zip((dataset_fullpapers, dataset_abstracts))
  training_dataset = training_dataset.shuffle(BUFFER_SIZE)
  training_dataset = training_dataset.batch(1)

  return training_dataset

In [0]:
TRAINING_PATH = "drive/My Drive/Tensorflow/Data/Training/"
TESTING_PATH = "drive/My Drive/Tensorflow/Data/Testing/"
training_fullpaper_filenames = tf.data.Dataset.list_files(TRAINING_PATH+'F*.txt')
training_abstract_filenames = tf.data.Dataset.list_files(TRAINING_PATH+'A*.txt')
training_dataset = load_textfiles_train(training_fullpaper_filenames,training_abstract_filenames)

testing_fullpaper_filenames = tf.data.Dataset.list_files(TESTING_PATH+'F*.txt')
testing_abstract_filenames = tf.data.Dataset.list_files(TESTING_PATH+'A*.txt')
testing_dataset = load_textfiles_train(testing_fullpaper_filenames,testing_abstract_filenames)

In [0]:
train(training_dataset, EPOCHS)

In [70]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
for fullpaper, abstract in testing_dataset.take(1):
  embedding_fullpaper_test,embedding_abstract_test = get_embedding_pair(fullpaper,abstract)
  prediction = generator(embedding_fullpaper_test, training=True)
print("Prediction for first test data:")
print(prediction)

Prediction for first test data:
tf.Tensor(
[[[[ 1.90219078e-02 -2.20442414e-02 -3.37352306e-01 ... -2.78456658e-01
     2.88418442e-01  2.99370229e-01]
   [ 1.86599508e-01  2.66008496e-01 -2.54604727e-01 ... -8.21608603e-01
    -5.62339723e-01 -9.78497043e-02]
   [ 4.14906412e-01 -8.14892799e-02  2.19248667e-01 ... -2.18538404e-01
    -4.59017217e-01  4.40648079e-01]
   ...
   [ 5.66907644e-01  1.17614321e-01  2.50003904e-01 ... -3.16822194e-02
    -1.79636329e-01 -9.41995680e-02]
   [-1.85234874e-01  5.05845547e-01 -4.34856981e-01 ... -4.73497093e-01
     1.23692535e-01  1.63708776e-01]
   [-3.53473544e-01 -2.38709182e-01 -1.82123810e-01 ... -4.21349287e-01
    -3.07353914e-01  1.12521604e-01]]

  [[-1.34058297e-01  3.31289321e-01 -7.31628001e-01 ... -3.26840997e-01
     2.24654570e-01  6.57139182e-01]
   [-4.70759779e-01 -5.14518976e-01  5.40225655e-02 ... -1.42396361e-01
     5.70397153e-02  5.85348845e-01]
   [ 1.42003447e-01  4.33402956e-01 -3.75856102e-01 ... -1.18926518e-01
    